In [1]:
import os
os.environ.get("SPARK_HOME")

'C:\\Xiaonan\\Courses\\Data engineering\\Spark\\spark-2.2.0-bin-hadoop2.7\\bin'

In [2]:
os.environ.get("HADOOP_HOME")

'C:\\Xiaonan\\Courses\\Data engineering\\Spark\\hadoop-2.7.1'

In [ ]:
import sys
sys.getdefaultencoding()
#reload(sys)
#sys.setdefaultencoding("utf-8")

In [3]:
import findspark
findspark.init()


In [ ]:
import pyspark
from pyspark.sql import SQLContext
from pyspark import SparkContext    

sc = SparkContext("local", "First App")
sqlContext = SQLContext(sc)

In [4]:
import configparser
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [5]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']


In [6]:
def create_spark_session():
    """Create spark session.
    Returns:
        spark (SparkSession) - spark session connected to AWS EMR cluster
    """
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark



In [7]:
def process_song_data(spark, input_data, output_data):
    '''
    Load song data from S3 and extracts the song and artist tables
    
    Keyword arguments:
    spark -- the spark session
    input_data -- the path of input song data from S3
    output_data -- the path of output data to S3
    
    '''
    # get filepath to song data file
    song_data = input_data + "song_data/*/*/*/*json"
    
    #song_data = "./data/song_data/*/*/*/*json"
    
    # read song data file
    df = spark.read.json(song_data).drop_duplicates()
    df.printSchema()

    # extract columns to create songs table
    songs_table = df.select("song_id", "title", "artist_id", "year", "duration").distinct()
    
    # write songs table to parquet files partitioned by year and artist
    songs_table.write.parquet(output_data + "songs_table.parquet", mode="overwrite", partitionBy=["year", "artist_id"])

    # extract columns to create artists table
    #artists_table = df.select("artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude").distinct()
    
    # write artists table to parquet files
    #artists_table.write.parquet(output_data + "artists_table.parquet", mode="overwrite")


In [8]:
def process_log_data(spark, input_data, output_data):
    '''
    Load log data from S3 and extracts the user, time and songplay tables
    
    Keyword arguments:
    spark -- the spark session
    input_data -- the path of input log data from S3
    output_data -- the path of output data to S3
    
    '''
    # get filepath to log data file
    log_data = input_data + "log_data/*/*/*json"

    # read log data file
    df = spark.read.json(log_data).drop_duplicates() 
    df.printSchema()
    
    # filter by actions for song plays
    df = df.filter(df.page=="NextSong")

    # extract columns for users table    
    #user_table = df.select("userId", "firstName", "lastName", "gender", "level").distinct()
    
    #user_table.show(1)
    
    # write users table to parquet files
    # user_table.write.parquet(output_data + "user_table.parquet", mode = "overwrite")

    # create timestamp column from original timestamp column
    #get_timestamp = udf(lambda ts: datetime.datetime.fromtimestamp(int(ts)/1000), T.TimestampType())
    #df = df.withColumn("start_time", get_timestamp(ts)) 
    
    # extract columns to create time table
    #time_table = df.withColumn("hour",     F.hour("start_time"))\
    #                .withColumn("day",     F.dayofmonth("start_time"))\
    #                .withColumn("week",    F.weekofyear("start_time"))\
    #                .withColumn("month",   F.month("start_time"))\
    #                .withColumn("year",    F.year("start_time"))\
    #                .withColumn("weekday", F.dayofweek("start_time"))\
    #                .select("ts","start_time","hour", "day", "week", "month", "year", "weekday")
    
    # write time table to parquet files partitioned by year and month
    #time_table=time_table.repartition(col("year"),col("month"))
    #time_table.write.parquet(output_data + "time_table.parquet", mode="overwrite", partitionBy = ["year", "month"])

    # read in song data to use for songplays table
    #song_df = spark.read.parquet(output_data + "songs_table.parquet")

    # extract columns from joined song and log datasets to create songplays table 
    #songplays_table = df.selectColumn("songplay_id", F.monotonically_increasing_id())\
    #                    .join(song_df, df.song == song_df.title, how="inner")\
    #                    .select("songplay_id", 
    #                           "start_time", 
    #                           col("userId").alias("user_id"),
    #                            "level", 
    #                            "song_id", 
    #                            "artist_id", 
    #                            col("sessionId").alias("session_id"), 
    #                            "location", 
    #                            col("userAgent").alias("user_agent")) 

    # write songplays table to parquet files partitioned by year and month
    # songplays_table=songplays_table.repartition(col("year"),col("month"))
    # songplays_table.write.parquet(output_data + "songplays_table.parquet", mode = "overwrite", partitionBy = ["year", "month"])


In [9]:
spark = create_spark_session()

In [10]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://udacity-xn/DataEngineering/"

In [ ]:
#process_song_data(spark, input_data, output_data) 

In [11]:
#song_data = "./data/song_data/*/*/*/*json"
    
# read song data file
#df = spark.read.json(song_data).drop_duplicates()
#df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [12]:
#songs_table = df.select("song_id", "title", "artist_id", "year", "duration").distinct()

In [13]:
#songs_table.show(5)

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOGOSOV12AF72A285E|   ¿Dónde va Chichi?|ARGUVEV1187B98BA17|1997|313.12934|
|SOTTDKS12AB018D69B|It Wont Be Christmas|ARMBR4Y1187B9990EB|   0|241.47546|
|SOBBUGU12A8C13E95D|Setting Fire to S...|ARMAC4T1187FB3FA4C|2004|207.77751|
|SOIAZJW12AB01853F1|          Pink World|AR8ZCNI1187B9A069B|1984|269.81832|
|SONYPOM12A8C13B2D7|I Think My Wife I...|ARDNS031187B9924F0|2005|186.48771|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows



In [ ]:
#sc=spark.sparkContext
#hadoop_conf=sc._jsc.hadoopConfiguration()

In [ ]:
#hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
#hadoop_conf.set("fs.s3n.awsAccessKeyId", os.environ["AWS_ACCESS_KEY_ID"])
#hadoop_conf.set("fs.s3n.awsSecretAccessKey", os.environ["AWS_SECRET_ACCESS_KEY"])

In [14]:
#songs_table.write.parquet(output_data + "songs_table.parquet", mode="overwrite", partitionBy=["year", "artist_id"])

In [15]:
song_data = input_data + "song_data/*/*/*/*json"

In [16]:
df = spark.read.json(song_data).drop_duplicates()
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [17]:
songs_table = df.select("song_id", "title", "artist_id", "year", "duration").distinct()

In [18]:
songs_table.write.parquet(output_data + "songs_table.parquet", mode="overwrite", partitionBy=["year", "artist_id"])

In [19]:
artists_table = df.select("artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude").distinct()

In [20]:
artists_table.write.parquet(output_data + "artists_table.parquet", mode="overwrite")